In [1]:
import pickle as pkl
import os
import pandas as pd

path = os.getcwd()
data_name = 'SMD'
data_file = os.path.join(path, 'processed', data_name)
train_data_ = open(os.path.join(data_file, 'machine-1-1_train.pkl'),'rb')
train_data = pd.DataFrame(pkl.load(train_data_))
test_data_ = open(os.path.join(data_file, 'machine-1-1_test.pkl'),'rb')
test_data = pd.DataFrame(pkl.load(test_data_))

test_data_label_ = open(os.path.join(data_file, 'machine-1-1_test_label.pkl'),'rb')
test_data_label = pd.DataFrame(pkl.load(test_data_label_),columns=['label'])

anomaly_num = len(test_data_label[test_data_label.label==1])
anomaly_num / len(test_data_label)

0.09459601811861372

In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_train_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data = pd.DataFrame(scaler.fit_transform(test_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.065678,0.068156,0.073256,0.0,0.925287,0.269886,0.0,0.031081,0.000000,...,0.0,0.004317,0.068036,0.048893,0.000386,0.000023,0.064432,0.064500,0.0,0.0
1,0.076087,0.080508,0.073742,0.075582,0.0,0.922414,0.270833,0.0,0.031081,0.000122,...,0.0,0.004317,0.070020,0.050437,0.000386,0.000011,0.065228,0.065224,0.0,0.0
2,0.065217,0.064619,0.069273,0.073256,0.0,0.919540,0.270833,0.0,0.030940,0.000366,...,0.0,0.004317,0.069684,0.055069,0.000386,0.000034,0.067111,0.067178,0.0,0.0
3,0.076087,0.048729,0.061452,0.069768,0.0,0.919540,0.268940,0.0,0.027250,0.000244,...,0.0,0.005756,0.073253,0.051467,0.000000,0.000023,0.066676,0.066744,0.0,0.0
4,0.076087,0.051907,0.060335,0.069768,0.0,0.925287,0.269886,0.0,0.030940,0.000244,...,0.0,0.004317,0.070932,0.051467,0.000386,0.000011,0.066604,0.066671,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.065217,0.051907,0.045810,0.046512,0.0,0.896551,0.252841,0.0,0.043571,0.000244,...,0.0,0.027339,0.046733,0.040144,0.000000,0.000011,0.042931,0.043000,0.0,0.0
28475,0.054347,0.025424,0.036871,0.043024,0.0,0.893678,0.252841,0.0,0.032501,0.000000,...,0.0,0.027339,0.047438,0.048893,0.000000,0.000045,0.046550,0.046619,0.0,0.0
28476,0.054347,0.080508,0.048044,0.046512,0.0,0.896551,0.253788,0.0,0.026114,0.000611,...,0.0,0.027339,0.046797,0.040144,0.000386,0.000034,0.043003,0.043000,0.0,0.0
28477,0.054347,0.056144,0.045810,0.045350,0.0,0.890805,0.252841,0.0,0.033210,0.000122,...,0.0,0.027339,0.041884,0.043232,0.000000,0.000034,0.039890,0.039959,0.0,0.0


In [3]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_train_data = pca.fit_transform(normalized_train_data)
pca_test_data = pca.fit_transform(normalized_test_data)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.6472445  0.23495755 0.03814479 0.01958584 0.01584372]
[67.19858   40.487465  16.313362  11.6895275 10.513673 ]


## Ensemble

In [4]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [5]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [6]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr,re,f1,roc

In [7]:
X_train = pca_test_data
X_test = pca_test_data
y_train = test_data_label
y_test = test_data_label

In [8]:
outliers_fraction = np.count_nonzero(y_train) / len(y_train)
outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
random_state = np.random.RandomState(42)
classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
    }

n_clf = len(classifiers)
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
p_labels = np.zeros([X_test.shape[0], n_clf])

i = 0
train_duration = 0
test_duration = 0
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train)
    t1 = time()
    test_sccore = clf.decision_function(X_test)
    t_t = time()
    test_mean_score = np.nanmean(test_scores)
    test_scores[np.isnan(test_scores)] = test_mean_score
    test_scores[:, i] = test_sccore
    p_labels[:, i] = pred_labels(y_test, test_sccore)
    i += 1
    print('Base detector %i is fitted for prediction' % i)
    train_duration += round(t1 - t0, ndigits=4)
    test_duration += round(t_t - t1, ndigits=4)
    
# standardize test score
test_scores_norm = standardizer(test_scores)
print(train_duration)

Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
Base detector 4 is fitted for prediction
79.72319999999999


## CNN

In [9]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from time import time

test_scores_norm = test_scores_norm.reshape((test_scores_norm.shape[0], test_scores_norm.shape[1], 1))
train_x, test_x, train_y, test_y = train_test_split(test_scores_norm, y_test, test_size=0.9, random_state=random_state)
#train_x, test_x, train_y, test_y = train_test_split(test_scores_norm[:,3], y_test, test_size=0.5, random_state=random_state)

In [10]:
def ensemble_cnn():
    df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
    enm_mlp_df = pd.DataFrame(columns=df_columns)

    t2 = time()
#     # Input - Layer
#     model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(4, 1)))
#     #model.add(Dropout(0.2))
#     model.add(LSTM(units=50, activation='relu', return_sequences=False))
#     #model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))

    model = Sequential()
    model.add(Conv1D(64, 2, activation="relu", input_shape=(4,1)))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.summary()
    # compiling the model
    model.compile(
        optimizer = "adam",
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
      
    results = model.fit(
     train_x, train_y,
     epochs= 100,
     batch_size = 20,
     validation_data = (train_x, train_y)
    )
    t3 = time()
    #print(results.history['val_accuracy'])
    #pred_test = model.predict(test_scores_norm[:,0]).reshape(len(test_scores_norm),1)
    pred_test = model.predict(test_scores_norm)
    #pred_test = model.predict(test_x)
    t4 = time()
    train_time = t3 - t2
    test_time = t4 - t3
    train_time_mlp =  round(train_duration+train_time, ndigits=4)
    test_time_mlp = round(test_duration+test_time, ndigits=4)
    
    pred_nn = []
    for each in pred_test:
        if each[0] > 0.5:
            pred_nn.append(1)
        else:
            pred_nn.append(0)
    #pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(test_y, pred_nn, pred_test)
    pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(y_test, pred_nn, pred_test)
    enm_mlp = pd.DataFrame([pr_mlp, re_mlp, f1_mlp, roc_mlp, train_time_mlp, test_time_mlp]).transpose()
    enm_mlp.columns = df_columns
    enm_mlp_df = pd.concat([enm_mlp_df, enm_mlp], axis=0)
    return enm_mlp_df, model

In [11]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
t_enm_mlp_df = pd.DataFrame(columns=df_columns)
for i in range(10):
    print(i)
    enm_mlp, model = ensemble_cnn()
    t_enm_mlp_df = pd.concat([t_enm_mlp_df, enm_mlp], axis=0, ignore_index=True)

0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 3, 64)             192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,249
Trainable params: 1,249
Non-trainable params: 0
_________________________________________________________________


2023-05-26 14:14:32.811098: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 14:14:33.045337: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
143/143 [==============================] - 15s 4ms/step - loss: 0.4785 - accuracy: 0.8974 - val_loss: 0.2267 - val_accuracy: 0.9125
Epoch 2/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2148 - accuracy: 0.9155 - val_loss: 0.2057 - val_accuracy: 0.9185
Epoch 3/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2106 - accuracy: 0.9098 - val_loss: 0.1994 - val_accuracy: 0.9224
Epoch 4/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1985 - accuracy: 0.9244 - val_loss: 0.1979 - val_accuracy: 0.9234
Epoch 5/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2195 - accuracy: 0.9141 - val_loss: 0.1973 - val_accuracy: 0.9259
Epoch 6/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1957 - accuracy: 0.9262 - val_loss: 0.1955 - val_accuracy: 0.9241
Epoch 7/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1925 - accuracy: 0.9309 - val_loss: 0.1955 - val_accuracy: 0.924

Epoch 58/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2024 - accuracy: 0.9266 - val_loss: 0.1849 - val_accuracy: 0.9305
Epoch 59/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1806 - accuracy: 0.9341 - val_loss: 0.1855 - val_accuracy: 0.9329
Epoch 60/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1952 - accuracy: 0.9299 - val_loss: 0.1870 - val_accuracy: 0.9301
Epoch 61/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2043 - accuracy: 0.9265 - val_loss: 0.1844 - val_accuracy: 0.9287
Epoch 62/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1956 - accuracy: 0.9301 - val_loss: 0.1835 - val_accuracy: 0.9301
Epoch 63/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1808 - accuracy: 0.9366 - val_loss: 0.1835 - val_accuracy: 0.9305
Epoch 64/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1718 - accuracy: 0.9350 - val_loss: 0.1840 - val_accuracy:

Epoch 8/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2144 - accuracy: 0.9228 - val_loss: 0.2018 - val_accuracy: 0.9182
Epoch 9/100
143/143 [==============================] - 0s 1ms/step - loss: 0.2046 - accuracy: 0.9199 - val_loss: 0.1938 - val_accuracy: 0.9266
Epoch 10/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1858 - accuracy: 0.9305 - val_loss: 0.1926 - val_accuracy: 0.9269
Epoch 11/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1996 - accuracy: 0.9261 - val_loss: 0.1942 - val_accuracy: 0.9259
Epoch 12/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1958 - accuracy: 0.9301 - val_loss: 0.1919 - val_accuracy: 0.9298
Epoch 13/100
143/143 [==============================] - 0s 1ms/step - loss: 0.1947 - accuracy: 0.9302 - val_loss: 0.1915 - val_accuracy: 0.9255
Epoch 14/100
143/143 [==============================] - 0s 2ms/step - loss: 0.2011 - accuracy: 0.9241 - val_loss: 0.1914 - val_accuracy: 0

...

Epoch 50/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1856 - accuracy: 0.9295 - val_loss: 0.1843 - val_accuracy: 0.9305
Epoch 51/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1876 - accuracy: 0.9242 - val_loss: 0.1840 - val_accuracy: 0.9290
Epoch 52/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1727 - accuracy: 0.9303 - val_loss: 0.1856 - val_accuracy: 0.9315
Epoch 53/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1811 - accuracy: 0.9318 - val_loss: 0.1849 - val_accuracy: 0.9301
Epoch 54/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1880 - accuracy: 0.9265 - val_loss: 0.1842 - val_accuracy: 0.9308
Epoch 55/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1890 - accuracy: 0.9251 - val_loss: 0.1839 - val_accuracy: 0.9298
Epoch 56/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1990 - accuracy: 0.9276 - val_loss: 0.1848 - val_accuracy:

143/143 [==============================] - 1s 4ms/step - loss: 0.4647 - accuracy: 0.9036 - val_loss: 0.2221 - val_accuracy: 0.9168
Epoch 2/100
143/143 [==============================] - 0s 2ms/step - loss: 0.2108 - accuracy: 0.9169 - val_loss: 0.2034 - val_accuracy: 0.9213
Epoch 3/100
143/143 [==============================] - 0s 2ms/step - loss: 0.2025 - accuracy: 0.9215 - val_loss: 0.1994 - val_accuracy: 0.9217
Epoch 4/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1987 - accuracy: 0.9196 - val_loss: 0.1988 - val_accuracy: 0.9241
Epoch 5/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1819 - accuracy: 0.9319 - val_loss: 0.1974 - val_accuracy: 0.9248
Epoch 6/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1973 - accuracy: 0.9287 - val_loss: 0.1970 - val_accuracy: 0.9255
Epoch 7/100
143/143 [==============================] - 0s 2ms/step - loss: 0.2002 - accuracy: 0.9223 - val_loss: 0.1957 - val_accuracy: 0.9248
Epoch 8/100

143/143 [==============================] - 0s 2ms/step - loss: 0.2066 - accuracy: 0.9289 - val_loss: 0.1858 - val_accuracy: 0.9269
Epoch 59/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1950 - accuracy: 0.9245 - val_loss: 0.1846 - val_accuracy: 0.9301
Epoch 60/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1888 - accuracy: 0.9281 - val_loss: 0.1846 - val_accuracy: 0.9322
Epoch 61/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1954 - accuracy: 0.9290 - val_loss: 0.1844 - val_accuracy: 0.9301
Epoch 62/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1867 - accuracy: 0.9300 - val_loss: 0.1842 - val_accuracy: 0.9301
Epoch 63/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1844 - accuracy: 0.9257 - val_loss: 0.1846 - val_accuracy: 0.9294
Epoch 64/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1876 - accuracy: 0.9289 - val_loss: 0.1839 - val_accuracy: 0.9308
Epoch

Epoch 8/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1897 - accuracy: 0.9303 - val_loss: 0.1934 - val_accuracy: 0.9259
Epoch 9/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1920 - accuracy: 0.9278 - val_loss: 0.1931 - val_accuracy: 0.9245
Epoch 10/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1999 - accuracy: 0.9237 - val_loss: 0.1935 - val_accuracy: 0.9231
Epoch 11/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1938 - accuracy: 0.9209 - val_loss: 0.1924 - val_accuracy: 0.9280
Epoch 12/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1939 - accuracy: 0.9228 - val_loss: 0.1957 - val_accuracy: 0.9213
Epoch 13/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1844 - accuracy: 0.9292 - val_loss: 0.1928 - val_accuracy: 0.9273
Epoch 14/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1931 - accuracy: 0.9223 - val_loss: 0.1938 - val_accuracy: 0

Epoch 65/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1986 - accuracy: 0.9300 - val_loss: 0.1846 - val_accuracy: 0.9287
Epoch 66/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1850 - accuracy: 0.9316 - val_loss: 0.1835 - val_accuracy: 0.9305
Epoch 67/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1917 - accuracy: 0.9248 - val_loss: 0.1846 - val_accuracy: 0.9301
Epoch 68/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1665 - accuracy: 0.9348 - val_loss: 0.1834 - val_accuracy: 0.9312
Epoch 69/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1839 - accuracy: 0.9308 - val_loss: 0.1832 - val_accuracy: 0.9308
Epoch 70/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1847 - accuracy: 0.9293 - val_loss: 0.1838 - val_accuracy: 0.9280
Epoch 71/100
143/143 [==============================] - 0s 2ms/step - loss: 0.1720 - accuracy: 0.9344 - val_loss: 0.1845 - val_accuracy:

In [12]:
print('Ensemble mlp')
metrics_name = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_mlp = []
std_mlp = []
for each in metrics_name:
    mean_mlp.append(round(t_enm_mlp_df[each].mean(), ndigits=4))
    std_mlp.append(round(t_enm_mlp_df[each].std(), ndigits=4))
print(mean_mlp)
print(std_mlp)
t_enm_mlp_df

Ensemble mlp
[0.8078, 0.7286, 0.7603, 0.9116, 105.4209, 40.7473]
[0.0031, 0.0117, 0.0076, 0.0008, 3.4649, 0.0363]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.8115,0.7229,0.7577,0.9101,114.6736,40.7947
1,0.8070,0.7328,0.7633,0.9113,102.1866,40.7021
2,0.8061,0.7301,0.7611,0.9106,102.4799,40.7131
3,0.8041,0.7436,0.7695,0.9126,105.4898,40.8130
4,0.8043,0.7393,0.7667,0.9123,105.1634,40.7579
5,0.8110,0.7076,0.7461,0.9115,105.1248,40.7378
6,0.8063,0.7284,0.7600,0.9114,103.9168,40.7542
7,0.8083,0.7422,0.7701,0.9124,104.9754,40.7561
8,0.8131,0.7152,0.7524,0.9124,104.5984,40.7070
9,0.8065,0.7234,0.7565,0.9118,105.6004,40.7372
